In [1]:
import pandas as pd
from datetime import datetime

In [2]:
import pandas as pd
import re

# Log dosyasının yolu
log_file_path = "example-access.log.txt"


In [3]:
# Zaman damgasını ve methodu ayrıştıralım :)
def parse_log_line(line):
    # Regex pattern ile log satırını ayırma (*StackOverFlow yardımı)
    pattern = r'(\S+) - - \[([^\]]+)\] "([^"]+)" (\d{3}) (\d+) "-" "([^"]+)" "-"'
    match = re.match(pattern, line)
    if match:
        ip, timestamp, request, status, size, user_agent = match.groups()
        try:
            method, url, protocol = request.split()
        except ValueError:
            method, url, protocol = ("", "", "")
            
        # Zaman damgası ve methodu doğru ayırma
        timestamp = f'[{timestamp}]'
        return [ip, timestamp, method, url, protocol, int(status), int(size), user_agent]
    return None

In [4]:
# Log dosyasını okuma
with open(log_file_path, 'r') as file:
    lines = file.readlines()

In [5]:
# Verileri ayrıştırma
data = [parse_log_line(line) for line in lines if parse_log_line(line) is not None]

In [6]:
# pandasla DataFrame'e dönüştürme
df = pd.DataFrame(data, columns=["IP", "timestamp", "Method", "URL", "Protocol", "StatusCode", "ResponseSize", "UserAgent"])
df.head()

,IP,timestamp,Method,URL,Protocol,StatusCode,ResponseSize,UserAgent
0,33.444.55.666,[11/Mar/2016:05:09:15 -0500],GET,/,HTTP/1.1,301,184,libwww-perl/6.05
1,33.444.55.666,[11/Mar/2016:05:09:16 -0500],GET,/,HTTP/1.1,400,37,libwww-perl/6.05
2,33.444.55.666,[11/Mar/2016:05:09:17 -0500],GET,/,HTTP/1.1,400,37,libwww-perl/6.05
3,33.444.55.666,[11/Mar/2016:05:09:18 -0500],GET,/,HTTP/1.1,400,37,libwww-perl/6.05
4,33.444.55.666,[11/Mar/2016:05:09:19 -0500],GET,/,HTTP/1.1,400,37,libwww-perl/6.05


#### yukarıda timestamp verilerinde -0500] bu kısmı anlayamadığım şekilde silemedim. Bu yüzden daha doğal hale getirmeye karar verdim

In [7]:
# Zaman damgalarını datetime'a dönüştüreyim
df['timestamp'] = pd.to_datetime(df['timestamp'], format='[%d/%b/%Y:%H:%M:%S %z]', errors='coerce')
df.head()

,IP,timestamp,Method,URL,Protocol,StatusCode,ResponseSize,UserAgent
0,33.444.55.666,2016-03-11 05:09:15-05:00,GET,/,HTTP/1.1,301,184,libwww-perl/6.05
1,33.444.55.666,2016-03-11 05:09:16-05:00,GET,/,HTTP/1.1,400,37,libwww-perl/6.05
2,33.444.55.666,2016-03-11 05:09:17-05:00,GET,/,HTTP/1.1,400,37,libwww-perl/6.05
3,33.444.55.666,2016-03-11 05:09:18-05:00,GET,/,HTTP/1.1,400,37,libwww-perl/6.05
4,33.444.55.666,2016-03-11 05:09:19-05:00,GET,/,HTTP/1.1,400,37,libwww-perl/6.05


In [8]:
# Gerekli Verilerin Seçimi
#aslında df dataframe'inde ki bütün sütunları dahil edicem(zaten verim az) ancak her zaman  durum böyle olmayabilir bu yüzden adettendir deyip ek bir df kuralım
df_selected = df[['IP', 'timestamp', "Method", 'URL', 'Protocol', 'StatusCode', "ResponseSize", "UserAgent"]]
df_selected.head()
#ayrıca şuanki verinin tamamını incelediğim de sadece bazı url bölümlerinde boşluklar olduğunu gördüm onları da ayıklamaya gerek görmedim

,IP,timestamp,Method,URL,Protocol,StatusCode,ResponseSize,UserAgent
0,33.444.55.666,2016-03-11 05:09:15-05:00,GET,/,HTTP/1.1,301,184,libwww-perl/6.05
1,33.444.55.666,2016-03-11 05:09:16-05:00,GET,/,HTTP/1.1,400,37,libwww-perl/6.05
2,33.444.55.666,2016-03-11 05:09:17-05:00,GET,/,HTTP/1.1,400,37,libwww-perl/6.05
3,33.444.55.666,2016-03-11 05:09:18-05:00,GET,/,HTTP/1.1,400,37,libwww-perl/6.05
4,33.444.55.666,2016-03-11 05:09:19-05:00,GET,/,HTTP/1.1,400,37,libwww-perl/6.05


In [9]:
# En çok erişilen sayfaları bulalım
top_pages = df_selected['URL'].value_counts().head(10)
print(f"En çok erişilen sayfalar:\n{top_pages}")

En çok erişilen sayfalar:
URL
/                     13
/test-endpoint         6
/manager/html          3
/admin                 2
/cgi-bin/php           1
/test/navs/global/     1
                       1
/test/seasons/         1
/test/shows/           1
/test/categories/      1
Name: count, dtype: int64


In [10]:
# En aktif IP adreslerini bulalım
top_ips = df_selected['IP'].value_counts().head(10)
print(f"En aktif IP adresleri:\n{top_ips}")

En aktif IP adresleri:
IP
111.222.333.444    10
33.444.55.666       9
222.333.444.555     8
444.555.666.77      3
777.888.99.111      1
888.99.11.22        1
999.111.222.333     1
55.666.777.888      1
666.777.8.99        1
112.223.334.445     1
Name: count, dtype: int64


In [11]:
# HTTP yanıt kodlarının dağılımını inceleyelim
Status_codes = df_selected['StatusCode'].value_counts()
print("\nHTTP Yanıt Kodlarının Dağılımı:")
print(Status_codes)


HTTP Yanıt Kodlarının Dağılımı:
StatusCode
301    14
400    11
200    11
Name: count, dtype: int64


In [12]:
#"How many HTTP requests were made? "
total_requests = df_selected['Method'].count()
print(f"Toplam HTTP isteği sayısı: {total_requests}")

Toplam HTTP isteği sayısı: 36


In [13]:
#En sık erişilen endpoint nedir?
endpoint_counts = df_selected['URL'].value_counts()

endpoint_counts_df = endpoint_counts.reset_index()
endpoint_counts_df.columns = ['Endpoint', 'AccessCount']

print(endpoint_counts_df)

                   Endpoint  AccessCount
0                         /           13
1            /test-endpoint            6
2             /manager/html            3
3                    /admin            2
4              /cgi-bin/php            1
5        /test/navs/global/            1
6                                      1
7            /test/seasons/            1
8              /test/shows/            1
9         /test/categories/            1
10            /test/events/            1
11             /test/items/            1
12             /test/staff/            1
13    /test/items/slug/bar/            1
14    /test/items/slug/foo/            1
15  /test/homepage/current/            1


In [14]:
# GET isteklerinin sayısını hesaplayın
get_requests_count = df_selected[df_selected['Method'] == 'GET'].shape[0]

print(f"Toplam GET isteği sayısı: {get_requests_count}")


Toplam GET isteği sayısı: 34


In [15]:
# 'libwww-perl/6.05' user agent'ı hangi IP adresleri tarafından kullanılmıştır?


specific_useragent_df = df_selected[df_selected['UserAgent'] == 'libwww-perl/6.05']

# Bu UserAgent'ın hangi IP adresleri tarafından kullanıldığı
ip_addresses = specific_useragent_df['IP'].unique()

print(f"'libwww-perl/6.05' UserAgent'ı tarafından kullanılan IP adresleri:")
for ip in ip_addresses:
    print(ip)


'libwww-perl/6.05' UserAgent'ı tarafından kullanılan IP adresleri:
33.444.55.666


In [16]:
#Kaç tane 400 hata kodu döndüren istek vardır?"

error_400_count = df_selected[df_selected['StatusCode'] == 400].shape[0]

print(f"400 hata kodu döndüren isteklerin sayısı: {error_400_count}")


400 hata kodu döndüren isteklerin sayısı: 11


In [17]:
# Kaç farklı user agent kullanılmıştır? 

unique_user_agents_count = df_selected['UserAgent'].nunique()

print(f"Farklı user agent sayısı: {unique_user_agents_count}")


Farklı user agent sayısı: 5


In [18]:
#En erken ve en geç yapılan istekler hangi saatlerde gerçekleşmiştir?

# En erken ve en geç yapılan isteklerin saati
earliest_request_time = df_selected['timestamp'].min()
latest_request_time = df_selected['timestamp'].max()

# Sonuçları yazdırın
print(f"En erken yapılan istek saati: {earliest_request_time}")
print(f"En geç yapılan istek saati: {latest_request_time}")


En erken yapılan istek saati: 2016-03-11 05:09:15-05:00
En geç yapılan istek saati: 2016-03-11 12:07:35-05:00


In [19]:
# Saat bazında isteklerin sayısında bir artış veya azalma eğilimi var mı?
df_selected['hour'] = df_selected['timestamp'].dt.hour

# Saat bazında istek sayısını gruplandırsam ...
hourly_request_counts = df_selected.groupby('hour').size()

print(hourly_request_counts)

hour
5      9
6      1
7      6
8      1
9      1
10    16
12     2
dtype: int64


In [20]:
# 404 hatası alan istekler var mı?

error_404_requests = df_selected[df_selected['StatusCode'] == 404]

# 404 hata kodu döndüren isteklerin sayısı
error_404_count = error_404_requests.shape[0]

if error_404_count > 0:
    print(f"404 hata kodu döndüren {error_404_count} adet istek bulunmaktadır.")
    print(error_404_requests)
else:
    print("404 hata kodu döndüren istek bulunmamaktadır.")

404 hata kodu döndüren istek bulunmamaktadır.


In [21]:
# 400 hatası veren endpoint'ler nelerdir?

error_400_requests = df_selected[df_selected['StatusCode'] == 400]

# 400 hatası döndüren endpoint
error_400_endpoints = error_400_requests['URL'].unique()

print("400 hatası döndüren endpoint'ler:")
for endpoint in error_400_endpoints:
    print(endpoint)

400 hatası döndüren endpoint'ler:
/
/admin



In [22]:
# kaç adet 400 hatası vardır? 

error_400_requests = df_selected[df_selected['StatusCode'] == 400]

error_400_count = error_400_requests.shape[0]

print(f"Toplam {error_400_count} adet 400 hata kodu döndüren istek bulunmaktadır.")


Toplam 11 adet 400 hata kodu döndüren istek bulunmaktadır.


In [23]:
# GET /admin isteğini gönderen IP adresleri hangileridir?"

admin_requests = df_selected[(df_selected['Method'] == 'GET') & (df_selected['URL'] == '/admin')]

# Bu isteği gönderen benzersiz IP adresleri
admin_ip_addresses = admin_requests['IP'].unique()


print("GET /admin isteğini gönderen IP adresleri:")
for ip in admin_ip_addresses:
    print(ip)


GET /admin isteğini gönderen IP adresleri:
888.99.11.22
999.111.222.333
